In [ ]:
import scipy.io
import re
import numpy as np
from collections import Counter
import os

In [21]:
mat = scipy.io.loadmat('MR001A.mat')

mat

{'__header__': b'MATLAB 5.0 MAT-file, Platform: PCWIN64, Created on: Tue Oct 22 13:59:07 2024',
 '__version__': '1.0',
 '__globals__': [],
 'matriz_resultante': array([[ -624.4,  -628.9,  -634.1, ...,  -535.2,  -533.2,  -531.5],
        [ -679.2,  -681.3,  -682.8, ...,  -644.7,  -642.6,  -641.2],
        [ -688.5,  -689.5,  -689.7, ...,  -623.5,  -621.2,  -619.3],
        ...,
        [ -678.2,  -680.5,  -681.9, ...,  -638.4,  -636.4,  -634.8],
        [ -659.9,  -660.7,  -660.9, ...,  -620.3,  -617.4,  -616.2],
        [ -602.7,  -622.8,  -672.3, ..., -1078.7,  -836.1,  -536.3]])}

In [22]:
dato = mat['matriz_resultante'].T
print(dato.shape)
dato

(96000, 32)


array([[ -624.4,  -679.2,  -688.5, ...,  -678.2,  -659.9,  -602.7],
       [ -628.9,  -681.3,  -689.5, ...,  -680.5,  -660.7,  -622.8],
       [ -634.1,  -682.8,  -689.7, ...,  -681.9,  -660.9,  -672.3],
       ...,
       [ -535.2,  -644.7,  -623.5, ...,  -638.4,  -620.3, -1078.7],
       [ -533.2,  -642.6,  -621.2, ...,  -636.4,  -617.4,  -836.1],
       [ -531.5,  -641.2,  -619.3, ...,  -634.8,  -616.2,  -536.3]])

In [23]:
dir = './registros/Tabla/Correctos'

directory = os.fsencode(dir)

X_data = np.array([])
c = 0
for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if filename.endswith(".mat"):
        mat_aux = scipy.io.loadmat(dir + '/' + filename)['matriz_resultante'].T
        #if mat_aux.shape[0] == 96000 and c == 0:
        if mat_aux.shape[0] == 96000:
            if c == 0:
                X_data = mat_aux
            else:
                X_data = np.concatenate((X_data, mat_aux), axis=0)
            c += 1

In [24]:
print(X_data.shape)
X_data

(1440000, 32)


array([[ -41.5,  -59.4,  -11. , ...,  -21.6,   -4.7, 1338.7],
       [ -40.7,  -59.2,  -11.8, ...,  -21.3,   -4.7, 1295.3],
       [ -39. ,  -58.6,  -11.6, ...,  -20.8,   -4.3, 1271.1],
       ...,
       [ -41.5,  -27. ,  -23. , ...,  -17.8,   -5.1,  -86.9],
       [ -41.4,  -27.2,  -22.8, ...,  -18.2,   -5.3,  -81.1],
       [ -41.1,  -27.5,  -22.7, ...,  -18.7,   -5.5,  -80.5]])

In [25]:
def generar_test(estr):
    with open('EventosAudio.txt', 'r') as f:
        y_data=[]
        n = 0
        for linea in f:
            if n != 0:
                # 0 para audio/no audio, 1 para anger/joy/neutral, 2 para anger/joy/neutral/no sonido
                cleaned_string = re.sub(r'[^a-zA-Z]', '', linea)
                if estr == 0:
                    for _ in range(500):
                        y_data.append('audio')
                    for _ in range(500):
                        y_data.append('no_audio')
                elif estr == 1:
                    for _ in range(1000):
                        y_data.append(cleaned_string)
                else:
                    for _ in range(500):
                        y_data.append(cleaned_string)
                    for _ in range(500):
                        y_data.append('no_audio')
            else:
                n += 1     
    return y_data*c

In [26]:
y_data = generar_test(1)

print(len(y_data))

conteo = {}
for elemento in y_data:
    conteo[elemento] = conteo.get(elemento, 0) + 1
print(conteo)

1440000
{'anger': 480000, 'joy': 480000, 'neutral': 480000}


In [39]:
def apply_sliding_window(X, y, window_size, step_size):
    y_res = []
    X_res = []
    posicion = 0
    stride = window_size - step_size
    total = len(y)
    
    while (posicion + window_size < total):
        y_aux = []
        X_aux = []
        
        for _ in range(window_size):
            y_aux.append(y[posicion])
            X_aux.append(X[posicion])
            posicion += 1
        
        valor_mas_comun = Counter(y_aux).most_common(1)
        y_res.append(valor_mas_comun[0][0])

        X_res.append(np.asarray(X_aux))

        posicion -= stride    
    
    return np.asarray(X_res), np.asarray(y_res)

In [41]:
windowed_data, windowed_y = apply_sliding_window(X_data, y_data, window_size=64, step_size=8)

print("Shape original:", X_data.shape)
print("Shape después de aplicar ventana:", windowed_data.shape)

print("Shape original:", np.array(y_data, dtype=str).shape)
print("Shape después de aplicar ventana:", windowed_y.shape)

Shape original: (1440000, 32)
Shape después de aplicar ventana: (179992, 64, 32)
Shape original: (1440000,)
Shape después de aplicar ventana: (179992,)


In [45]:
with open('dataset_X_sliding_window.npy', 'wb') as f:
    np.save(f, windowed_data)

In [46]:
with open('dataset_y_sliding_window.npy', 'wb') as f:
    np.save(f, windowed_y)